Notebook used to calculate the brain connectivity matrix based on FDG-PET baseline data used as input for the graph neural networks. The connectivity graph will be estimated based on Gaussian graphical models (Graphical Lasso).

In [ ]:
import os
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
import warnings
import networkx as nx
from typing import Union
from pathlib import Path
from tqdm import tqdm
from sklearn.covariance import GraphicalLasso
from IPython.display import display
from datetime import datetime

In [ ]:
# base data directory
BASE_DATA_PATH = Path(os.path.join('..',  'data'))

# FDG-PET data
PATH_TO_FDG_DATA = BASE_DATA_PATH / 'mounts' / 'v1' / '20240428_fdg_cross_sectional.parquet'

# path to split information used to prevent information about test data from being leaked in the calculation 
# of connectivity networks
PATH_TO_SPLIT_INFO = BASE_DATA_PATH / 'splits' / 'v1' / '20240428_longitudinal_generated20240428.parquet'

# directory where the generated connectivity matrices will be exported to
PATH_TO_OUTPUT = BASE_DATA_PATH / 'graphical_lasso' / 'v1'

In [ ]:
def boostrapGraphicalLasso(
    X: pd.DataFrame,
    alphas: Union[list, np.ndarray],
    niter: int,
    n_samples: int,
    replacement: bool,
    maxiter: int,
    n_jobs: int = 1
):
    """ Subroutine used to fit the Graphical Lasso model by applying boostrap to extract 
    the BIC value and a stability estimate of the selected edges.
    """
    def __worker__(
        _X: pd.DataFrame,
        _alpha: float,
        _niter: int,
        _n_samples: int,
        _replacement: bool,
        _maxiter: int
    ):
        precisions = []
        bics = []
        loglike = []
        determinants = []
        pen_term = []
        eigenvalues = []
        condition = []
        n_edges = []
        curr_iter_ = 0
        with tqdm(total=_niter) as pbar:
            while len(bics) < _niter:
                # generate the sample data
                X_sample = _X.iloc[np.random.choice(range(len(_X)), size=_n_samples, replace=_replacement)]

                try:
                    with warnings.catch_warnings(record=True) as w:
                        warnings.simplefilter("always")
                        glasso = GraphicalLasso(
                            alpha=_alpha,
                            mode='cd',
                            covariance='precomputed',
                            max_iter=100,
                            assume_centered=True,
                        ).fit(X_sample.cov())
                            
                except Exception as ex:
                    curr_iter_ += 1
                    if curr_iter_ > _maxiter:
                        print('(1) Exception (alpha={}): {}'.format(_alpha, ex))
                        break
                    else:
                        continue
                    
                # calculate the log likelihood of the data (positive better)
                # Sklearn assumes that the sample size is constant and does not include 
                # it in the equation. For this reason it is necessary to multiply by N
                log_likelihood = glasso.score(X_sample) * len(X_sample)
                if np.isinf(log_likelihood):
                    curr_iter_ += 1
                    if curr_iter_ > _maxiter:
                        print('(2) Maximum number of iterations reached (alpha={})'.format(_alpha))
                        break
                    else:
                        continue

                # save the log-likelihood
                loglike.append(log_likelihood)

                # save the determinant of the precision matrix
                determinants.append(
                    np.linalg.det(glasso.precision_)
                )
                
                # save the penalization term of the objective function
                pen_term.append(
                    np.sum(np.abs(glasso.precision_))
                )

                # save the eigenvalues
                eigenvalues.append(
                    np.linalg.eig(glasso.precision_)[0]
                )

                # save the condition number (euclidean norm(
                condition.append(
                    np.linalg.cond(glasso.precision_, p=None)
                )

                # get the precision matrix
                precision = glasso.precision_
                
                # binarize the precision matrix
                zero_mask = np.isclose(precision, 0)
                precision[np.where(zero_mask)] = 0
                precision[np.where(~zero_mask)] = 1
                
                # calculate the number of parameters
                num_params = np.sum(np.triu(precision != 0, k=0))

                # get the number of edges
                n_edges.append(num_params - len(precision))   # off-diagonal elements
                
                # calculate the bic (inverting the sign to match sklearn implementation framework)
                bic = 2 * log_likelihood - num_params * np.log(len(X_sample))

                # save the precision matrix and bic
                precisions.append(precision)
                bics.append(bic)
            
                curr_iter_ += 1
                pbar.update(1)
                
                if curr_iter_ > _maxiter:
                    print('(3) Maximum number of iterations reached (alpha={})'.format(_alpha))
                    break
            
        # get the mean precision matrix
        mean_precision = np.stack(precisions).mean(axis=0)
    
        return {
            'alpha': _alpha,
            'mean_precision': mean_precision,
            'bic': np.array(bics),
            'll': np.array(loglike),
            'det': np.array(determinants),
            'l_1': np.array(pen_term),
            'eigen': np.stack(eigenvalues),
            'n_edges': np.array(n_edges),
            'condition': np.array(condition)
            
        }

    if n_jobs == 1:
        glasso_out = [
            __worker__(
                _X=X,
                _alpha=alpha,
                _niter=niter,
                _n_samples=n_samples,
                _replacement=replacement,
                _maxiter=maxiter
            ) for alpha in alphas
        ]
    else:
        glasso_out = joblib.Parallel(backend='loky', n_jobs=n_jobs)(
            joblib.delayed(__worker__)(
                _X=X,
                _alpha=alpha,
                _niter=niter,
                _n_samples=n_samples,
                _replacement=replacement,
                _maxiter=maxiter
            ) for alpha in alphas
        )
    
    return {
        'alpha': np.array([e['alpha'] for e in glasso_out]), 
        'bic': np.array([e['bic'] for e in glasso_out]),
        'll': np.array([e['ll'] for e in glasso_out]),
        'det': np.array([e['det'] for e in glasso_out]),
        'l_1': np.array([e['l_1'] for e in glasso_out]),
        'eigen': np.array([e['eigen'] for e in glasso_out]),
        'n_edges': np.array([e['n_edges'] for e in glasso_out]),
        'condition': np.array([e['condition'] for e in glasso_out]),
        'mean_precision': np.array([e['mean_precision'] for e in glasso_out])
    }

# Data loading

In [ ]:
# load the input data and select the variables of intereset for the target population
df = pd.read_parquet(PATH_TO_FDG_DATA)
split_df = pd.read_parquet(PATH_TO_SPLIT_INFO)
print(f'Data shape (initial): {df.shape[0]}')

# prevent possible data leak from the test data
df = df.loc[
  ~df.index.get_level_values('subject_id').isin(
      split_df.loc[split_df['split'] == 'test'].index.get_level_values('subject_id').values
  ) 
].copy()

# remove dementia subjects
df = df.loc[
    df['diagnosis'].isin(['control', 'mci', 'dementia'])
].copy()
print(f'Data shape (no dementia): {df.shape[0]}')

# select baseline information
df = df.groupby('subject_id').nth(0)
print(f'Data shape (baseline): {df.shape[0]}')

# save diagnostic information
diag_df = df[['diagnosis']].copy()

# select average FDG SUVR values removing vermis and cerebellum
df = df[[
    c for c in df.columns if re.match('^aal_pet_fdg.*mean', c)
  ]].copy()

print(f'Data shape (variable selection): {df.shape}')

In [ ]:
# process the input data
X = df.copy()

th = 1.2   # HACK. Avoid creating meta-rois
rois_to_merge = []
correlations = []
X_meta = X.copy()
for i in range(X.shape[1]):
    for j in range(i+1, X.shape[1]):
        corr = X.iloc[:, [i, j]].corr().iloc[0, 1]
        correlations.append(corr)
        if abs(corr) > th:
            # display the information
            print(list(X.iloc[:, [i, j]].columns), round(corr, 3))

            # select the names
            r1, r2 = X.columns[i], X.columns[j]

            # save the ROIs
            rois_to_merge.append((r1, r2))

# create the meta-ROI
if len(rois_to_merge) > 0:
    # create a graph to explore the relationships
    G = nx.Graph()
    G.add_edges_from(rois_to_merge)
    
    # get the subgraphs
    subgraphs = list(nx.connected_components(G))
    
    # display the graph
    nx.draw(G, with_labels=True, node_color='lightblue', edge_color='gray', node_size=500, font_size=8)
    plt.show()
    
    subgraph_hash = {}
    for idx, elements in enumerate(subgraphs):
        subgraph_hash[idx] = elements
        X_meta['META_%d' % idx] = X_meta[list(elements)].mean(axis=1)
        X_meta = X_meta.drop(columns=list(elements))

# standarize the data 
X_meta = (X_meta - X_meta.mean()) / X_meta.std()
X_meta.shape

In [ ]:
# explore the correlation between brain regions
fig, ax = plt.subplots(figsize=(6, 3.5))
ax.hist(correlations, bins=30)
ax.set_title('Correlations', size=15)
ax.grid(alpha=0.2, color='black')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
sns.heatmap(
    X_meta.cov(),
    vmin=-1, vmax=1,
    cmap='Spectral_r',
    ax=ax,
)
plt.show()

# Graphical lasso

In [ ]:
# bootstrapping analysis of lambda values
output = boostrapGraphicalLasso(
    X=X_meta,
    alphas=np.linspace(0.1, 0.8, 100),
    niter=500,
    n_samples=int(len(X) * 0.75),
    replacement=False,
    maxiter=1000,
    n_jobs=20
)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 2.5))
axes[0].plot(
    output['alpha'],    
    output['condition'].mean(axis=1),
)
axes[0].set_title(r'Mean cond (euc)', size=14)

axes[1].plot(
    output['alpha'],    
    output['condition'].std(axis=1),
)
axes[1].set_title(r'Std cond (euc)', size=14)

for ax in axes.flatten():
    for pos in ['top', 'bottom', 'right', 'left']:
        ax.spines[pos].set_visible(False)
    ax.grid(alpha=0.3, color='black')
    ax.set_xlabel('$\lambda$')

plt.subplots_adjust(wspace=0.3)
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 2.5))
axes[0].plot(
    output['alpha'],    
    np.log(output['det']).mean(axis=1),
)
axes[0].set_title(r'Mean $log(|\Theta|)$', size=14)

axes[1].plot(
    output['alpha'],    
    np.log(output['det']).std(axis=1),
)
axes[1].set_title(r'Std $log(|\Theta|)$', size=14)

for ax in axes.flatten():
    for pos in ['top', 'bottom', 'right', 'left']:
        ax.spines[pos].set_visible(False)
    ax.grid(alpha=0.3, color='black')
    ax.set_xlabel('$\lambda$')

plt.subplots_adjust(wspace=0.3)
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 2.5))
axes[0].plot(
    output['alpha'],    
    output['n_edges'].mean(axis=1),
)
axes[0].set_title(r'Mean number of edges', size=14)

axes[1].plot(
    output['alpha'],    
    output['n_edges'].std(axis=1),
)
axes[1].set_title(r'Std number of edges', size=14)

for ax in axes.flatten():
    for pos in ['top', 'bottom', 'right', 'left']:
        ax.spines[pos].set_visible(False)
    ax.grid(alpha=0.3, color='black')
    ax.set_xlabel('$\lambda$')
    
plt.subplots_adjust(wspace=0.3)
plt.show()

In [ ]:
# According to the implementation of sklearn higher values of log-likelihodd are better
#
fig, ax = plt.subplots(figsize=(5, 3))
ax.plot(
    output['alpha'],    
    output['bic'].mean(axis=1),
    lw=2,
    label='BIC'
)
ax.plot(
    output['alpha'],    
    output['ll'].mean(axis=1),
    lw=2,
    label='LL'
)
ax.plot(
    output['alpha'],    
    (2*output['ll'] - output['n_edges'] * np.log(int(len(X) * 0.75)) - 
     4 * output['n_edges'] * np.log(X_meta.shape[1]) * .5).mean(axis=1),
    lw=2,
    label='eBIC ($\gamma=0.5$)'
)

for pos in ['top', 'bottom', 'right', 'left']:
    ax.spines[pos].set_visible(False)
ax.grid(alpha=0.3, color='black')
ax.set_xlabel('$\lambda$')
    
plt.xlabel('$\lambda$')
plt.legend()
plt.show()

# optimal lambda according to eBIC
output['alpha'][np.argmax(
    (2*output['ll'] - output['n_edges'] * np.log(int(len(X) * 0.75)) - 
     4 * output['n_edges'] * np.log(X_meta.shape[1]) * .5).mean(axis=1)
)]

In [ ]:
# fit various graphical lasso models for different values of lambda selected on the basis of the above analyses
lambda_values = [0.1, 0.2, 0.3, 0.4]
final_conn_matrices = {}
for lambda_ in lambda_values:
    print(f'----- Processing lambda: {lambda_:.3f}')

    # create and fit the final glasso model using all available data
    final_glasso = GraphicalLasso(
        alpha=lambda_,
        mode='cd',
        covariance='precomputed',
        max_iter=500,
        assume_centered=True,
    ).fit(X_meta.cov())

    print('Model log-likelihood: {:.2f}'.format(final_glasso.score(X_meta) * len(X_meta)))
    print('Precision matrix determinant: {:.2f}'.format(np.linalg.det(final_glasso.precision_)))
    print('Number of edges: {:.0f}'.format((~np.isclose(final_glasso.precision_, 0.0)).sum() / 2 - X_meta.shape[1]))

    # create the final connectivity matrix
    final_conn = pd.DataFrame(
        (~np.isclose(final_glasso.precision_, 0)).astype(int),
        columns=X_meta.columns,
        index=X_meta.columns)

    # add the ROIs that were grouped into Meta-ROIs
    if len(rois_to_merge) > 0:
        for k, rois in subgraph_hash.items():
            meta_roi = 'META_%d' % k
            for roi in rois:
                final_conn.loc[roi] = final_conn.loc[meta_roi]
                final_conn[roi] = final_conn[meta_roi]
        
            # remove meta ROI
            final_conn = final_conn.drop(index=[meta_roi], columns=[meta_roi])
        
            # add connections between ROIs
            final_conn.loc[list(rois), list(rois)] = 1
        
    # remove diagonal elements
    for i in range(len(final_conn)):
        final_conn.iloc[i, i] = 0

    # fix variable names
    var_hash = {
        k: k.replace('aal_pet_fdg_', '').replace('_mean', '') 
        for k in final_conn.columns}
    final_conn.columns = [var_hash[c] for c in final_conn.columns]
    final_conn.index = [var_hash[c] for c in final_conn.index]

    # check if the remaining graph is connected
    G = nx.from_pandas_adjacency(final_conn)
    assert nx.is_connected(G),\
        f'Graph associated to lambda {lambda_:.3f} contains unconnected nodes.'

    # calculate some graph theory metrics
    graph_metrics = pd.concat([
        pd.DataFrame([nx.degree_centrality(G)], index=['Centrality']).T,
        pd.DataFrame([nx.eigenvector_centrality(G)], index=['Eigenvector centrality']).T,
        pd.DataFrame([nx.betweenness_centrality(G)], index=['Betweenness centrality']).T,
        pd.DataFrame([nx.average_neighbor_degree(G)], index=['Neighboor degree']).T
    ], axis=1)

    # display the graph metrics for the top 10
    gmetric_df_head = []
    for gmetric in [
        'Centrality', 'Eigenvector centrality', 'Betweenness centrality', 'Neighboor degree'
    ]:
        sub_df_ = graph_metrics.sort_values(by=gmetric, ascending=False)[[gmetric]].head(10)
        sub_df_.index.names = [f'ROI- {gmetric}']
        sub_df_ = sub_df_.reset_index()
        gmetric_df_head.append(sub_df_)
        
    display(pd.concat(gmetric_df_head, axis=1).round(decimals=3))

    # save the connectivity matrix 
    final_conn_matrices[lambda_] = final_conn

    print('\n')

In [ ]:
# export the connectivity matrices
curr_date = datetime.now().strftime('%Y%m%d')
for lambda_, conn_matrix in final_conn_matrices.items():
    conn_matrix.to_parquet(
        PATH_TO_OUTPUT / f'{curr_date}_lambda_{lambda_:.3f}.parquet'
    )